In [74]:
import openeo
from openeo.rest.datacube import PGNode, THIS
from openeo.processes import *
import math

In [75]:
# backend = "openeo.cloud"
backend = "https://openeo-dev.eodc.eu/openeo/1.1.0/"
conn = openeo.connect(backend)
conn = conn.authenticate_oidc()

Authenticated using refresh token.


In [83]:
def sig0_process(bbox = [3, 43, 17, 50], geom = None, temporal__extent = ["2022-09-02", "2022-09-30"]):
    spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3]}
    collection      = 'SENTINEL1_SIG0_20M'
    bands           = ["VV", "VH"]
    properties = {"sat:orbit_state": lambda x: eq(x, "ascending")}

    l2a = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent, properties=properties).reduce_dimension(dimension="time", reducer="median") # .add_dimension(name="t", label="0", type="temporal")
    
    if not geom:
        geom = l2a.process("load_vector_cube", {"URL": "https://raw.githubusercontent.com/openEOPlatform/SRR3_notebooks/main/notebooks/resources/UC8/vector_data/target_canopy_cover_60m_WGS84/target_canopy_cover_WGS84_60m.geojson"})
    l2a_agg = l2a.aggregate_spatial(geometries=geom, reducer="median") 
    from openeo.rest.datacube import THIS

    fit = l2a_agg.process("fit_regr_random_forest", {"predictors": THIS, "target": geom, "predictors_vars": bands, "target_var": "target_canopy_cover"})
    fit_saved = fit.save_result(format="GeoJSON") 
    
    return fit_saved 

In [72]:
def boa_process(bbox = [3, 43, 17, 50], geom = None, temporal_extent = ["2020-02-02", "2020-02-28"]):
    spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3]}
    collection      = 'boa_sentinel_2'
    bands           = ["B02", "B03", "B04"] 

    l2a = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent).reduce_dimension(dimension="t", reducer="median")
    
    if not geom:
        geom = l2a.process("load_vector_cube", {"URL": "https://raw.githubusercontent.com/openEOPlatform/SRR3_notebooks/main/notebooks/resources/UC8/vector_data/target_canopy_cover_60m_WGS84/target_canopy_cover_WGS84_60m.geojson"})
    
    
    l2a_agg = l2a.aggregate_spatial(geometries=geom, reducer="median")
    from openeo.rest.datacube import THIS
    fit = l2a_agg.process("fit_regr_random_forest", {"predictors": THIS, "target": geom, "predictors_vars": bands, "target_var": "target_canopy_cover"})
    fit_saved = fit.save_result(format="GeoJSON")
    
    
    return fit_saved

In [84]:
rf_sig0 = sig0_process(bbox, geom, temporal_extent)

In [86]:
job = rf_sig0.create_job(title = "UC8-1")
jobId = job.job_id
job.start_job()
print("Jobs submitted: ", jobId)

Jobs submitted:  f0c0a364-d313-4dcd-9cd8-242dbcb15c02


In [88]:
job

<BatchJob job_id='f0c0a364-d313-4dcd-9cd8-242dbcb15c02'>

In [89]:
def sig0_prediction(bbox = [14.3, 46.5, 14.4, 46.6]):
    spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3]}
    temporal_extent = ["2022-09-02", "2022-09-05"]
    collection      = 'SENTINEL1_SIG0_20M'
    bands           = ["VV", "VH"]
    properties = {"sat:orbit_state": lambda x: eq(x, "ascending")}

    l2a = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent, properties=properties).reduce_dimension(dimension="time", reducer="median")

    job_id = "f0c0a364-d313-4dcd-9cd8-242dbcb15c02"
    model = conn.load_stac(url = f'{backend}/jobs/{job_id}/results')
    predict = l2a.apply_dimension("predict_random_forest", dimension = "bands", context = {"model": job_id})
    predict_nc = predict.save_result(format="NetCDF")
    
    return predict_nc

In [4]:
def prediction(bbox):
    spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3]}
    temporal_extent = ["2020-02-02", "2020-02-28"] 
    collection      = 'boa_sentinel_2'
    bands           = ["B02", "B03", "B04"] 

    l2a = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent).reduce_dimension(dimension="t", reducer="median") # .add_dimension(name="t", label="0", type="temporal")
    job_id = "e3b51932-ee6e-436a-8973-6b449eb5ccd1"
    model = conn.load_stac(url = f'{backend}/jobs/{job_id}/results')
    predict = l2a.apply_dimension("predict_random_forest", dimension = "bands", context = {"model": job_id})
    predict_nc = predict.save_result(format="NetCDF")
    
    return predict_nc

In [90]:
pg = sig0_prediction()

job = pg.create_job(title = "UC8-1")
jobId = job.job_id
job.start_job()
print("Jobs submitted: ", jobId)

Jobs submitted:  cc63c9dd-01d6-4200-af16-f93c5eac02d4


In [52]:
job

<BatchJob job_id='644115fa-4a2a-4a9e-9cf5-06e8b4eda199'>

In [1]:
job.get_results().download_files("./UC8/")